In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pkg_resources

import debvader

2022-04-27 16:34:07.636985: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /pbs/throng/lsst/users/bbiswas/miniconda3/envs/debvader/lib/:
2022-04-27 16:34:07.637027: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Download and format data

Download dataset that will be used for training. It has been generated using the code in https://github.com/BastienArcelin/dc2_img_generation and the stamps are generated using the XXX function see notebook XXX. The size of the stamp is then fixed to 59x59 pixels.

In [2]:
data_folder_path = pkg_resources.resource_filename('debvader', "data/")
image_path = os.path.join(data_folder_path + 'dc2_imgs/imgs_dc2.npy')
images = np.load(image_path, mmap_mode = 'c')

In [3]:
images.shape

(10, 59, 59, 6)

Separate training data and labels to feed the VAE and deblender. You should be careful that the number of filter in the data for trainng correspond to the number of bands considered for the network (default is six).

In [4]:
training_data_vae = np.array((images[:5], images[:5]))
validation_data_vae = np.array((images[5:], images[5:]))

training_data_deblender = np.array((images[:5], images[:5]))
validation_data_deblender = np.array((images[5:], images[5:]))

### Train the VAE and deblender with architecture from the debvader paper (https://arxiv.org/abs/2005.12039)

In [5]:
from debvader.train import train_deblender
hist_vae, hist_deblender, net = train_deblender(survey_name=None,
                                                  from_survey = None, 
                                                  vae_epochs=5, 
                                                  deblender_epochs=5,
                                                  batch_size=5,
                                                  latent_dim=10,
                                                  training_data_vae = training_data_vae, 
                                                  validation_data_vae = validation_data_vae, 
                                                  training_data_deblender = training_data_deblender, 
                                                  validation_data_deblender = validation_data_deblender,   
                                                  initial_weights_path=None,
                                                  lr_scheduler_epochs=30)

2022-04-27 16:34:15.764340: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /pbs/throng/lsst/users/bbiswas/miniconda3/envs/debvader/lib/:
2022-04-27 16:34:15.764417: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-27 16:34:15.764463: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jns-bbiswas.cc.in2p3.fr): /proc/driver/nvidia/version does not exist
2022-04-27 16:34:15.764813: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-27 1

in cropping
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Instructions for updating:
`batch_shape` and `event_shape` args are deprecated. Please use `tfd.Sample`, `tfd.Independent`, and broadcasted parameters of the base distribution instead. For example, replace `tfd.TransformedDistribution(tfd.Normal(0., 1.), tfb.Exp(), batch_shape=[2, 3], event_shape=[4])` with `tfd.TransformedDistrbution(tfd.Sample(tfd.Normal(tf.zeros([2, 3]), 1.),sample_shape=[4]), tfb.Exp())` or `tfd.TransformedDistribution(tfd.Independent(tfd.Normal(tf.zeros([2, 3, 4]), 1.), reinterpreted_batch_ndims=1), tfb.Exp())`.
Instructions for updating:
`batch_shape` and `event_shape` args are deprecated. Please use `tfd.Sample`, `tfd.Independent`, and broadcasted parameters of the base distribution instead. For example, replace `tfd.TransformedDistribution(tfd.Normal(0., 1.), tfb.Exp(), batch_shape=[2, 3], event_shape=[4])` with `tfd.TransformedDistrbution(tfd.Sample(tfd.Normal(tf

2022-04-27 16:34:16.746441: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 59, 59, 6)]       0         
_________________________________________________________________
encoder (Functional)         (None, 65)                5709881   
_________________________________________________________________
latent_space (MultivariateNo ((None, 10), (None, 10))  0         
_________________________________________________________________
decoder (Functional)         (None, 59, 59, 6)         3316044   
Total params: 9,025,925
Trainable params: 9,025,913
Non-trainable params: 12
_________________________________________________________________
None

Start the training
Epoch 1/5
1/1 [==============================] - ETA: 0s - loss: 850977344.0000 - mse: 0.0863 - kl_metric: 0.0178
Epoch 00001: val_mse improved from inf to 0.11126, saving model to /pbs/throng/lsst/users/bbiswas/debv

### Dataset with a different number of filters

The number of available filters is different for each survey. For example, five filters are available for DES data. To change that you should specify it in the train_deblender function, as well as if the channels appear last or first in the data array.

In [6]:
training_data_vae.shape

(2, 5, 59, 59, 6)

Here the channels appear last in the data array. So the channel_first option in the train_deblender function is set as False (default setting).

In [7]:
# Remember to modify the input shape accordingly
hist_vae, hist_deblender, net = debvader.train.train_deblender(survey_name="des",
                                                  input_shape=[59, 59, 5],
                                                  from_survey = None, 
                                                  vae_epochs=250, 
                                                  deblender_epochs=250,
                                                  batch_size=5,
                                                  latent_dim=10,
                                                  training_data_vae = training_data_vae, 
                                                  validation_data_vae = validation_data_vae, 
                                                  training_data_deblender = training_data_deblender, 
                                                  validation_data_deblender = validation_data_deblender,   
                                                  initial_weights_path=None)

in cropping
The number of bands in the data does not correspond to the number of filters in the network. Correct this before starting again.


ValueError: 

We need to change the training data format. For example just train on the five fist channels.

In [8]:
training_data_vae_deslike = np.array((images[:5,:,:,:5], images[:5,:,:,:5]))
validation_data_vae_deslike = np.array((images[5:,:,:,:5], images[5:,:,:,:5]))

training_data_deblender_deslike = np.array((images[:5,:,:,:5], images[:5,:,:,:5]))
validation_data_deblender_deslike = np.array((images[5:,:,:,:5], images[5:,:,:,:5]))

Now try the training:

In [9]:
data_folder_path = pkg_resources.resource_filename('debvader', "data/")
path_output = os.path.join(data_folder_path, 'weights/dc2/not_normalised/')
latest = tf.train.latest_checkpoint(path_output)

Warning: we cannot use the weights from the network trained on DC2 as the data has six channels in that case. We need to set the from_survey option to None.

In [10]:
hist_vae, hist_deblender, net = debvader.train.train_deblender(survey_name="des",
                                                  input_shape=[59, 59, 5],
                                                  from_survey = None, 
                                                  vae_epochs=5, 
                                                  deblender_epochs=5,
                                                  batch_size=5,
                                                  latent_dim=10,
                                                  training_data_vae = training_data_vae_deslike, 
                                                  validation_data_vae = validation_data_vae_deslike, 
                                                  training_data_deblender = training_data_deblender_deslike, 
                                                  validation_data_deblender = validation_data_deblender_deslike,   
                                                  initial_weights_path=None)

in cropping
Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 59, 59, 5)]       0         
_________________________________________________________________
encoder (Functional)         (None, 65)                5709589   
_________________________________________________________________
latent_space (MultivariateNo ((None, 10), (None, 10))  0         
_________________________________________________________________
decoder (Functional)         (None, 59, 59, 5)         3315466   
Total params: 9,025,055
Trainable params: 9,025,045
Non-trainable params: 10
_________________________________________________________________
None

Start the training
Epoch 1/5
1/1 [==============================] - ETA: 0s - loss: 439312832.0000 - mse: 0.0609 - kl_metric: 0.0262
Epoch 00001: val_mse improved from inf to 0.09277, saving model to /pbs/throng/lsst/users/